In [24]:
import sys
sys.path.insert(0,'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
import cv2
import torchvision
from torch import nn
import numpy as np
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
from torch.autograd import Variable
import os
import pandas as pd
import seaborn as sns
from dataloaders import get_train_transforms, get_val_transforms, get_triplet_dataloader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

path_data = 'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
device = torch.cuda.is_available()

# استخراج ویژگی‌ها با استفاده از مدل Siamese
def generate_embeddings(data_loader, model):
    with torch.no_grad():
        model.eval()
        embeddings = []
        labels = []
        device = torch.cuda.is_available()
        for batch_imgs, batch_labels in data_loader:
            
            if device:
                batch_imgs = batch_imgs.cuda()
            
            # تقسیم تصویر به پچ‌ها
            batch_size = batch_imgs.size(0)
            num_patches = (224 // 16) * (224 // 16)
            embed_dim = 768
            patches = batch_imgs.unfold(2, 16, 16).unfold(3, 16, 16)
            patches = patches.contiguous().view(batch_size, 3, -1, 16, 16)
            patches = patches.permute(0, 2, 1, 3, 4).contiguous().view(batch_size * num_patches, 3, 16, 16)
            
            # استخراج امبدینگ پچ‌ها
            patch_embeddings = model.get_embedding(patches)
            patch_embeddings = patch_embeddings.view(batch_size, num_patches, -1)
            
            # تغییر اندازه امبدینگ پچ‌ها به 768
            patch_embeddings = nn.Linear(patch_embeddings.size(-1), embed_dim)(patch_embeddings)
            
            # اضافه کردن CLS token
            cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim)).expand(batch_size, -1, -1)
            patch_embeddings = torch.cat((cls_token, patch_embeddings), dim=1)
            print(f"patch_embeddings size:{patch_embeddings.shape}")
            # اضافه کردن position embedding
            pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
            patch_embeddings = patch_embeddings + pos_embed
            
            embeddings.append(patch_embeddings.cpu().numpy())
            labels.append(batch_labels.numpy())
            print(f"embeddings size:{embeddings.shape}")
    return np.concatenate(embeddings), np.concatenate(labels)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
siamese_model = torch.load("C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_Mobilenet_15Shot.h5", map_location=torch.device(device))
siamese_model.eval()

# تعریف مدل سفارشی ViT
class RefinedViT(nn.Module):
    def __init__(self, original_vit_model, embedding_dim, num_classes):
        super(RefinedViT, self).__init__()
        self.num_patches = (224 // 16) * (224 // 16)
        self.embed_dim = 768
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim,
            nhead=12,
            dim_feedforward=3072,
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=12)
        
        # لایه‌های نهایی
        self.norm = nn.LayerNorm(self.embed_dim)
        self.fc = nn.Linear(self.embed_dim, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # Transformer Encoder
        x = self.transformer_encoder(x)
        
        # استفاده از CLS token برای طبقه‌بندی
        x = x[:, 0]
        x = self.norm(x)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

train_data = torchvision.datasets.ImageFolder(root=path_data + '/train/', transform=get_val_transforms())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32)

test_data = torchvision.datasets.ImageFolder(root=path_data + '/test/', transform=get_val_transforms())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

# بارگذاری مدل ViT از Hugging Face
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
vit_model.classifier = torch.nn.Linear(vit_model.config.hidden_size, 15)

embedding_dim = vit_model.config.hidden_size
model = RefinedViT(vit_model, embedding_dim, num_classes=15)

# استخراج ویژگی‌ها از داده‌های آموزش
train_embeddings, train_labels = generate_embeddings(train_loader, siamese_model)
# استخراج ویژگی‌ها از داده‌های تست
test_embeddings, test_labels = generate_embeddings(test_loader, siamese_model)

# تبدیل داده‌ها به تنسور
X_train, y_train = train_embeddings, train_labels
X_val, y_val = test_embeddings, test_labels

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# انتقال مدل به دستگاه CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.is_available():
    model.cuda()

# تنظیمات loss function و optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# تنظیمات آموزش
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000)

# آموزش
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

torch.save(model, "C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/RVIT_improvedpatchmobilenet.h5")

# ارزیابی مدل
model.eval()
with torch.no_grad():
    val_outputs = model(X_val_tensor.to(device))
    _, predicted = torch.max(val_outputs.data, 1)

# ذخیره مدل
torch.save(model.state_dict(), "C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/RVIT_State_improvedpatchmobilenet.h5")

# محاسبه دقت و امتیاز F1
accuracy = accuracy_score(y_val_tensor.cpu(), predicted.cpu())
f1 = f1_score(y_val_tensor.cpu(), predicted.cpu(), average='weighted')
precision = precision_score(y_val_tensor.cpu(), predicted.cpu(), average='macro')
recall = recall_score(y_val_tensor.cpu(), predicted.cpu(), average='macro')

print(f'Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')


C:\Users\Mey\AppData\Local\Temp\ipykernel_1544\361712038.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  siamese_model = torch.load("C:/Users/Mey/Documents/PlantDisease

patch_embeddings size:torch.Size([32, 197, 768])


AttributeError: 'list' object has no attribute 'shape'